<a href="https://colab.research.google.com/github/Natasha617/DZ_NLP/blob/DZ_3/DZ_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашняя работа №3 Шафорост Н.В.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp "/content/drive/MyDrive/магистратура/NLP/train.csv" /content/data.csv

In [ ]:
!pip install fasttext

In [97]:
import numpy as np
from sklearn.manifold import TSNE
import pandas as pd
import re
import nltk
import gensim
import fasttext
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import plotly.express as px
from sklearn.metrics import classification_report

In [50]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [57]:
data = pd.read_csv("data.csv", encoding='utf-8')
data

,sms,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...
5569,This is the 2nd time we have tried 2 contact u...,1
5570,Will ü b going to esplanade fr home?\n,0
5571,"Pity, * was in mood for that. So...any other s...",0
5572,The guy did some bitching but I acted like i'd...,0


# Очистка данных

In [58]:
new_df=[]
for d in data['sms']:
    new_df.append(d.split(" "))

new_df
#все числа и URL-адреса были преобразованы в строки как NUMBERи URL соответственно
no_trash = [] # здесь будут данные без "URL" И "NUMBER"
for li in new_df:
    no = []
    for word in li:
      word = re.sub('[^a-zA-Zа-яА-Я]+', ' ', word)
      if word != 'URL' and word != "NUMBER":
        no.append(word)
    no_trash.append(no)

unique_stops = set(stopwords.words('english'))

# создаём пустой список без стоп-слов
no_stops = []
for li in no_trash:
    no_stop = []
    for word in li:
      word = word.lower()
      if word not in unique_stops and word.isalpha():
        no_stop.append(word)
    no_stops.append(no_stop)

lemmatizer = WordNetLemmatizer()

# и пустой список для слов после лемматизации
lemmatized = []
for li in no_stops:
    lemma = []
    for word in li:
      word = lemmatizer.lemmatize(word)
      lemma.append(word)
    lemmatized.append(lemma)

data['sms'] = lemmatized

data

,sms,label
0,"[go, jurong, available, bugis, n, great, world...",0
1,"[ok, joking, wif, u]",0
2,"[free, entry, wkly, comp, win, fa, cup, final,...",1
3,"[u, dun, say, early, u, c, already]",0
4,"[nah, think, go, life, around]",0
...,...,...
5569,"[time, tried, contact, u, pound, claim, call, ...",1
5570,"[b, going, esplanade, fr]",0
5571,[mood],0
5572,"[guy, bitching, acted, like, interested, buyin...",0


In [66]:
data['label'].value_counts()

0    4827
1     747
Name: label, dtype: int64

In [60]:
RANDOM_STATE = 123

# **Word2Vec SkipGram / CBOW**

In [61]:
%%time

model_skipGram = gensim.models.Word2Vec(
    sentences=data['sms'],
    vector_size=256,
    window=3,
    min_count=10,
    sg=1, # Training algorithm: 1 for skip-gram
    hs=0,
    negative=5,
    epochs=25,
    seed=RANDOM_STATE,

)

CPU times: user 3.17 s, sys: 28.1 ms, total: 3.19 s
Wall time: 3.03 s


In [62]:
print(list(model_skipGram.wv.index_to_key))

['u', 'call', 'get', 'ur', 'go', 'like', 'know', 'got', 'come', 'free', 'want', 'good', 'text', 'send', 'time', 'need', 'love', 'going', 'r', 'one', 'still', 'ok', 'txt', 'n', 'see', 'think', 'tell', 'dont', 'take', 'new', 'mobile', 'reply', 'make', 'day', 'back', 'please', 'claim', 'say', 'stop', 'give', 'home', 'hope', 'phone', 'pls', 'hi', 'much', 'happy', 'today', 'let', 'thing', 'great', 'sorry', 'ask', 'c', 'hey', 'really', 'im', 'co', 'da', 'last', 'meet', 'e', 'week', 'miss', 'well', 'would', 'work', 'friend', 'number', 'win', 'every', 'contact', 'find', 'feel', 'oh', 'pick', 'way', 'right', 'nokia', 'dear', 'keep', 'said', 'msg', 'sent', 'cant', 'night', 'service', 'even', 'wat', 'tone', 'cash', 'b', 'also', 'could', 'yeah', 'prize', 'went', 'gud', 'many', 'message', 'next', 'gonna', 'someone', 'show', 'buy', 'anything', 'sure', 'per', 'dun', 'thanks', 'always', 'guy', 'wan', 'yes', 'told', 'wait', 'later', 'customer', 'waiting', 'year', 'wish', 'something', 'lol', 'leave', 'f

In [63]:
model_skipGram.wv.most_similar(positive=['good'])

[('afternoon', 0.8705199956893921),
 ('checking', 0.8634317517280579),
 ('morning', 0.8599331378936768),
 ('sleep', 0.8469310402870178),
 ('thank', 0.8228162527084351),
 ('wonderful', 0.8202673196792603),
 ('fine', 0.8159391283988953),
 ('sweet', 0.8139969706535339),
 ('hope', 0.8128907084465027),
 ('lovely', 0.8119279146194458)]

In [64]:
model_skipGram.wv.most_similar(positive=['free'])

[('camcorder', 0.8568751811981201),
 ('term', 0.8102431893348694),
 ('unlimited', 0.800847589969635),
 ('auction', 0.7954052090644836),
 ('player', 0.7938231229782104),
 ('latest', 0.7916122078895569),
 ('credit', 0.7849550843238831),
 ('content', 0.7682967782020569),
 ('entry', 0.7600165009498596),
 ('ringtone', 0.759759783744812)]

In [67]:
model_skipGram.wv.doesnt_match(['like', 'good', 'going'])

'like'

In [68]:
model_skipGram.wv.doesnt_match(['take', 'make', 'stop'])

'take'

# CBOW

In [69]:
%%time

model_cbow = gensim.models.Word2Vec(
    sentences=data['sms'],
    vector_size=256,
    window=3,
    min_count=10,
    sg=0, # Training algorithm: 0 for cbow
    hs=0,
    negative=5,
    epochs=25,
    seed=RANDOM_STATE,

)

CPU times: user 1.93 s, sys: 32 ms, total: 1.96 s
Wall time: 2.49 s


In [70]:
model_cbow.wv.most_similar(['take', 'make'])

[('crave', 0.970413863658905),
 ('first', 0.96686851978302),
 ('everything', 0.9660127758979797),
 ('plan', 0.9657533168792725),
 ('read', 0.9655759930610657),
 ('making', 0.965548038482666),
 ('smoke', 0.9649391770362854),
 ('least', 0.9647969603538513),
 ('shit', 0.9637686610221863),
 ('kind', 0.9635578393936157)]

Вывод - для введенных слов получаются достаточно логичные ответы

# FastText

In [17]:
#data_str =[]
#for d in data["sms"]:
 # data_str.append(" ".join(d))

#with open('data.txt', 'w') as f:
 #   f.writelines(data_str)

In [ ]:
#model_fasttext = fasttext.train_unsupervised('data.txt', wordNgrams=3, dim=256)

In [72]:
%%time

model_fasttext = gensim.models.FastText(
    sentences=data['sms'],
    min_n=3,
    max_n=5,
    vector_size=256,  # default = 100
    window=3,  # default = 5
    min_count=10,
    sg=1,  # Training algorithm: 1 for skip-gram; otherwise CBOW
    hs=0,
    #  If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
    negative=5,  # If > 0, negative sampling will be used, if set to 0, no negative sampling is used.
    epochs=25,  # Number of iterations (epochs) over the corpus
    seed=RANDOM_STATE,
)

CPU times: user 9.3 s, sys: 4.21 s, total: 13.5 s
Wall time: 12.3 s


In [75]:
model_fasttext.wv.most_similar(positive=['secret'])

[('make', 0.8268299698829651),
 ('tried', 0.8061160445213318),
 ('contact', 0.7928506731987),
 ('asked', 0.7462389469146729),
 ('think', 0.7439468502998352),
 ('second', 0.7357189655303955),
 ('dating', 0.7275363206863403),
 ('looking', 0.722825825214386),
 ('wil', 0.7206500768661499),
 ('school', 0.7157709002494812)]

In [76]:
model_fasttext.wv.most_similar(positive=['natalia'])

[('national', 0.9566741585731506),
 ('pobox', 0.9356648921966553),
 ('ntt', 0.9203218221664429),
 ('rate', 0.902862012386322),
 ('operator', 0.8995065689086914),
 ('fantasy', 0.8945485353469849),
 ('box', 0.8929278254508972),
 ('collection', 0.8842521905899048),
 ('valued', 0.8840823769569397),
 ('credit', 0.8774592280387878)]

Вывод: выводит похожие слова даже для тех слов, которых не было в словаре

# Сравнение

In [77]:
model_skipGram.wv.most_similar(positive= ["school"] )

[('thinking', 0.9066758751869202),
 ('pretty', 0.8472338318824768),
 ('gave', 0.8441844582557678),
 ('leaving', 0.8411120176315308),
 ('yet', 0.8394403457641602),
 ('knew', 0.8393863439559937),
 ('working', 0.834182620048523),
 ('online', 0.8328015804290771),
 ('driving', 0.832524299621582),
 ('booked', 0.832129955291748)]

In [78]:
model_cbow.wv.most_similar(positive=['school'])

[('thinking', 0.994768500328064),
 ('knew', 0.9945111870765686),
 ('eh', 0.9935082197189331),
 ('decided', 0.9933621287345886),
 ('yet', 0.9933050870895386),
 ('telling', 0.9932901263237),
 ('leaving', 0.9930906295776367),
 ('finished', 0.9929855465888977),
 ('pretty', 0.9926241636276245),
 ('happened', 0.9925897717475891)]

In [79]:
model_fasttext.wv.most_similar(positive=['school'])

[('booked', 0.9000470638275146),
 ('food', 0.8934179544448853),
 ('pretty', 0.8902953863143921),
 ('mm', 0.8871684074401855),
 ('haha', 0.885444700717926),
 ('decided', 0.8816037178039551),
 ('promise', 0.877407431602478),
 ('used', 0.8763375282287598),
 ('else', 0.869695782661438),
 ('point', 0.8664596676826477)]

 Skip-gram  и CBOW по контексту выдает более оптимальные варианты, а Fasttext схожие по морфемам.

# Визуализация полученных векторов


In [24]:
import matplotlib.pyplot as plt
def make_vizual_model(model):
    words = list(model.wv.key_to_index.keys())

    word_vectors = np.array([model.wv.get_vector(word) for word in words])

    tsne = TSNE(n_components=3, random_state=0)
    word_vectors_3d = tsne.fit_transform(word_vectors)



    plt.scatter(tsne[:, 0], tsne[:, 1])
    for i, word in enumerate(model.wv.index2word):
      plt.annotate(word, xy=(tsne[i, 0], tsne[i, 1]))
    plt.show()



# Обучение LogisticRegression


In [26]:
unk_word_vector = np.random.rand(256) # вектор для слов, котрые не встречались в тексте

In [88]:
def mean_vector(doc, model):
    sentence_vector = [model.wv.get_vector(word) if model.wv.has_index_for(word) else unk_word_vector for word in doc]
    if not sentence_vector:
      return np.zeros(256)
    return np.mean(sentence_vector, axis=0)

In [89]:
data['vector_skip_gram'] = data['sms'].apply(lambda x: mean_vector(x, model_skipGram))
data['vector_cbow'] = data['sms'].apply(lambda x: mean_vector(x, model_cbow))
data['vector_fasttext'] = data['sms'].apply(lambda x: mean_vector(x, model_fasttext))

In [83]:
data.head()

,sms,label,vector_skip_gram,vector_cbow,vector_fasttext
0,"[go, jurong, available, bugis, n, great, world...",0,"[0.34200708301605026, 0.37384535165490945, 0.4...","[0.34187436968498547, 0.39815631212809177, 0.5...","[0.3853458926246874, 0.29606088156644, 0.39887..."
1,"[ok, joking, wif, u]",0,"[0.11672714349873733, 0.12931414339031871, 0.3...","[0.1450308412040062, 0.3080731180430561, 0.450...","[0.07733026889212799, 0.13311722054418024, 0.2..."
2,"[free, entry, wkly, comp, win, fa, cup, final,...",1,"[0.17451335352454797, 0.33637768226520665, 0.2...","[0.17930628088145048, 0.32321901362496236, 0.1...","[0.29088309718138067, 0.12681605118899414, 0.3..."
3,"[u, dun, say, early, u, c, already]",0,"[-0.13074645, 0.047014184, 0.22826147, -0.0866...","[-0.12945297, 0.21594672, 0.33778098, -0.13029...","[-0.17439173, -0.028593889, 0.021341603, 0.087..."
4,"[nah, think, go, life, around]",0,"[0.1831883296151787, 0.35734538180225967, 0.33...","[0.15508923248994647, 0.33023878313073396, 0.4...","[0.20532458198863565, 0.15665972563841818, 0.2..."


In [90]:
x_train, x_test, y_train, y_test = train_test_split(data['vector_skip_gram'], data['label'],random_state=RANDOM_STATE)

In [91]:
model_log_reg = LogisticRegression()
model_log_reg.fit(list(x_train), list(y_train))

LogisticRegression()

In [98]:
pred_skip_gram = model_log_reg.predict(list(x_test))
print(classification_report(y_test, pred_skip_gram))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93      1190
           1       1.00      0.17      0.29       204

    accuracy                           0.88      1394
   macro avg       0.94      0.58      0.61      1394
weighted avg       0.89      0.88      0.84      1394



In [92]:
x_train, x_test, y_train, y_test = train_test_split(data['vector_cbow'], data['label'],random_state=RANDOM_STATE)

In [94]:
# Создание и обучение модели логистической регрессии на эмбеддингах cbow
model_cbow_regression = LogisticRegression()
model_cbow_regression.fit(list(x_train), y_train)

LogisticRegression()

In [100]:
pred_cbow = model_cbow_regression.predict(list(x_test))
print(classification_report(y_test, pred_cbow))

              precision    recall  f1-score   support

           0       0.86      1.00      0.92      1190
           1       0.79      0.05      0.10       204

    accuracy                           0.86      1394
   macro avg       0.82      0.53      0.51      1394
weighted avg       0.85      0.86      0.80      1394



In [95]:
x_train, x_test, y_train, y_test = train_test_split(data['vector_fasttext'], data['label'],random_state=RANDOM_STATE)

In [96]:
# Создание и обучение модели логистической регрессии на эмбеддингах fast text
model_fasttext_regression = LogisticRegression()
model_fasttext_regression.fit(list(x_train), y_train)

LogisticRegression()

In [101]:
pred_fasttext = model_fasttext_regression.predict(list(x_test))
print(classification_report(y_test, pred_fasttext))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1190
           1       0.84      0.74      0.79       204

    accuracy                           0.94      1394
   macro avg       0.90      0.86      0.88      1394
weighted avg       0.94      0.94      0.94      1394



ВЫВОД:
Лучше всего себя показал Fasttext
А CBOW показал самый плохой результат